In [5]:
corpus=[
		    'food is very good',
		    'awesome food and nice quality',
		    'food is not good',
		    'poor quality no toppings'
	  ]

In [11]:
from keras.preprocessing.text import Tokenizer
tk=Tokenizer()
tk.fit_on_texts(corpus)
print(tk.word_index)
print(tk.index_word)

{'food': 1, 'is': 2, 'good': 3, 'quality': 4, 'very': 5, 'awesome': 6, 'and': 7, 'nice': 8, 'not': 9, 'poor': 10, 'no': 11, 'toppings': 12}
{1: 'food', 2: 'is', 3: 'good', 4: 'quality', 5: 'very', 6: 'awesome', 7: 'and', 8: 'nice', 9: 'not', 10: 'poor', 11: 'no', 12: 'toppings'}


In [12]:
seqes=tk.texts_to_sequences(corpus)
print(seqes)

[[1, 2, 5, 3], [6, 1, 7, 8, 4], [1, 2, 9, 3], [10, 4, 11, 12]]


In [23]:
from keras.preprocessing.sequence import pad_sequences

In [29]:
pad_seqes=pad_sequences(seqes,padding='pre')
pad_seqes.shape
pad_seqes

array([[ 0,  1,  2,  5,  3],
       [ 6,  1,  7,  8,  4],
       [ 0,  1,  2,  9,  3],
       [ 0, 10,  4, 11, 12]], dtype=int32)

In [39]:
from tensorflow.keras.layers import Embedding
vocab_size=len(tk.word_index)
word_dim=3
seq_len=pad_seqes.shape[1]
emd=Embedding(input_dim=vocab_size+1,output_dim=word_dim,input_length=seq_len)

In [40]:
word_emd=emd(1) #returns word enbedding of word whose position is 1 in vocabulary

In [41]:
import numpy as np
word_emd_seq=emd(np.array([ 0,  1,  2,  5,  3]))

In [42]:
word_emd_seques=emd(pad_seqes)
print(word_emd_seques.shape) #samples,seq_length,word_dimd


(4, 5, 3)


# How to combine word embedding using Embedding layer with RNN/LSTM/GRU 

In [61]:
from tensorflow.keras.layers import SimpleRNN,Dense,GlobalAveragePooling1D,LSTM,GRU
from tensorflow.keras.models import Sequential

In [62]:
corpus=[
		    'food is very good',
		    'awesome food and nice quality',
		    'food is not good',
		    'poor quality no toppings'
	  ]    

y=np.array([1,1,0,0])

tk=Tokenizer()
tk.fit_on_texts(corpus)
pad_seqes=pad_sequences(seqes,padding='pre')

vocab_size=len(tk.word_index)
word_dim=3
seq_len=pad_seqes.shape[1]

model=Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=word_dim,input_length=seq_len))
#model.add(SimpleRNN(10,input_shape=(word_dim,seq_len),return_sequences=True))
model.add(LSTM(10,input_shape=(word_dim,seq_len),return_sequences=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(2,activation='softmax'))
model.compile('adam','sparse_categorical_crossentropy',['accuracy'])
model.fit(pad_seqes,y,epochs=5)



Epoch 1/5
1/1 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.7500
Epoch 2/5
1/1 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 1ms/step - loss: 0.6916 - accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - 0s 1ms/step - loss: 0.6914 - accuracy: 0.5000
Epoch 5/5
1/1 [==============================] - 0s 1ms/step - loss: 0.6913 - accuracy: 0.5000


In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
df=pd.read_csv("drive/My Drive/dataset_dl/sentiment/Restaurant_Reviews.txt",sep='\t')
X=df.Review
y=df.Liked

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=10)

tk=Tokenizer()
tk.fit_on_texts(X_train)

seq_train=tk.texts_to_sequences(X_train)
pad_seqes_train=pad_sequences(seq_train,padding='pre')

vocab_size=len(tk.word_index)
word_dim=50
seq_len=pad_seqes_train.shape[1]

seq_test=tk.texts_to_sequences(X_test)
pad_seqes_test=pad_sequences(seq_test,padding='pre',truncating='pre',maxlen=seq_len)

model=Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=word_dim,input_length=seq_len))
#model.add(SimpleRNN(20,input_shape=(word_dim,seq_len),return_sequences=True))
#model.add(LSTM(10,input_shape=(word_dim,seq_len),return_sequences=True))
model.add(GRU(10,input_shape=(word_dim,seq_len),return_sequences=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(10))
model.add(Dense(2,activation='softmax'))
model.compile('adam','sparse_categorical_crossentropy',['accuracy'])
model.fit(pad_seqes_train,y_train,epochs=20)


Epoch 1/20
24/24 [==============================] - 0s 12ms/step - loss: 0.6934 - accuracy: 0.5080
Epoch 2/20
24/24 [==============================] - 0s 12ms/step - loss: 0.6851 - accuracy: 0.5480
Epoch 3/20
24/24 [==============================] - 0s 12ms/step - loss: 0.6687 - accuracy: 0.6307
Epoch 4/20
24/24 [==============================] - 0s 12ms/step - loss: 0.6177 - accuracy: 0.7813
Epoch 5/20
24/24 [==============================] - 0s 12ms/step - loss: 0.5008 - accuracy: 0.8013
Epoch 6/20
24/24 [==============================] - 0s 12ms/step - loss: 0.3608 - accuracy: 0.8627
Epoch 7/20
24/24 [==============================] - 0s 12ms/step - loss: 0.2500 - accuracy: 0.9200
Epoch 8/20
24/24 [==============================] - 0s 12ms/step - loss: 0.2013 - accuracy: 0.9253
Epoch 9/20
24/24 [==============================] - 0s 12ms/step - loss: 0.1611 - accuracy: 0.9600
Epoch 10/20
24/24 [==============================] - 0s 11ms/step - loss: 0.1278 - accuracy: 0.9573
Epoch 11/

In [67]:
model.evaluate(pad_seqes_test,y_test)

8/8 [==============================] - 0s 3ms/step - loss: 0.7820 - accuracy: 0.7360


[0.7820084691047668, 0.7360000014305115]

In [68]:
samples=['food quality is awesome','cheap quality','will not come again']
seq_samples=tk.texts_to_sequences(samples)
pad_seqes_samples=pad_sequences(seq_samples,padding='pre',truncating='pre',maxlen=seq_len)
np.argmax(model.predict(pad_seqes_samples),axis=-1)


array([1, 0, 0])